# NIFTI Label Alignment Script

In [ ]:
import os
import nibabel as nib
import numpy as np

def align_label_to_image(image_path, label_path, output_folder):
    try:

        img = nib.load(image_path)
        lbl = nib.load(label_path)

        img_data = img.get_fdata()
        lbl_data = lbl.get_fdata()

  
        if lbl_data.shape != img_data.shape:
            print(f"Resizing {os.path.basename(label_path)} from {lbl_data.shape} to {img_data.shape}")
            lbl_data = lbl_data[:, :, :img_data.shape[2]] 

        lbl_data = np.flip(lbl_data, axis=1)  
        lbl_data = np.flip(lbl_data, axis=2) 


        corrected_affine = img.affine.copy()

     
        fixed_label = nib.Nifti1Image(lbl_data, corrected_affine, img.header)


        output_path = os.path.join(output_folder, os.path.basename(label_path))
        nib.save(fixed_label, output_path)
        print(f"Aligned label saved: {output_path}")

    except Exception as e:
        print(f"Error processing {label_path}: {e}")

def process_labels(image_folder, label_folder, output_folder):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

  
    for label_filename in os.listdir(label_folder):
        if label_filename.endswith(".nii.gz"):
            image_path = os.path.join(image_folder, label_filename.replace(".nii.gz", "_0000.nii.gz"))
            label_path = os.path.join(label_folder, label_filename)

            if os.path.exists(image_path):
                align_label_to_image(image_path, label_path, output_folder)
            else:
                print(f"Skipping {label_filename}: No matching image found in {image_folder}")

image_folder = r"[base_path]\imagesTr"   
label_folder = r"[base_path]\transformed_label"  
output_folder = r"[base_path]\transformed_images" 

process_labels(image_folder, label_folder, output_folder)